# After getting files

## Insert into database

In [1]:
from pathlib import Path
try:
    dataset
except:
    print('getting dataset location')
    dataset = Path('dataset').absolute()


getting dataset location


In [2]:
import os
os.chdir(dataset)

js_dir = dataset / 'javascript'


In [3]:
import xml.etree.ElementTree as ET

In [4]:
import sqlite3, json
from tqdm import tqdm
from contextlib import contextmanager

train = js_dir / 'final' / 'jsonl' / 'train'
os.chdir(train)

@contextmanager
def get_cursor(database_name='rsn_train'):
    with sqlite3.connect(database_name) as conn:
        yield conn.cursor()




In [5]:
import regex

def simple_parse_xml(content, nested):
    pattern = r'\s*<([^\s]*?)>\s*'
    open = re.search(pattern, content)
    if not open:
        return [content]
    [open_start, open_end] = open.span()
    before = content[:open_start]
    tag = open.group(1)
    inner_and_after = content[open_end:]
    close = re.search(f'</{re.escape(tag)}>', inner_and_after)
    if close:
        [close_start, close_end] = close.span()
    else:
        close = re.search(pattern, inner_and_after)
        if close:
            [close_start, _] = close.span()
        else:
            close_start = len(inner_and_after)
        close_end = close_start
    inner = inner_and_after[:close_start]
    after = inner_and_after[close_end:]
    inner = simple_parse_xml(inner, nested) if nested else inner
    return [[before, tag, inner], *simple_parse_xml(after, nested)]

def atom_to_re(s):
    tokens = [token.strip() for token in re.split(r'\s+|(?=\W)|(?<=\W)', s.strip()) if token]
    escaped = [regex.escape(token, special_only=True) for token in tokens]
    return r'\s*' + r'\s*'.join(escaped) + r'\s*'

def str_to_re(s):
    codes = re.split(r'\s*(?://[^\n]*(?:\n|$)|/\*.*?\*/|\.{3,})\s*', s,  flags=re.DOTALL)
    return '(?:.*?)'.join(atom_to_re(code) for code in codes)

def node_to_re(node, c):
    if type(node) == str:
        return str_to_re(node), []
    before, tag, content = node
    before_re = str_to_re(before)
    c[0] += 1
    open_gr = c[0]
    content_re, content_tags = make_regex(content, c)
    c[0] += 1
    close_gr = c[0]
    open_re = '\s*(|<'+re.escape(tag)+'>)\s*'
    close_re = '\s*(|</'+re.escape(tag)+'>)\s*'
    return before_re+open_re+content_re+close_re, [(tag, open_gr, close_gr, content_tags)]


def make_regex(tree, c):
    regs, tags = zip(*(node_to_re(node, c) for node in tree))
    return re.sub(r'(\\s\*)+', r'\\s*', ''.join(regs)), [t for tag in tags for t in tag] 



In [ ]:
import json, re
omitted = []
output = []
for text_file in js_dir.glob('thread2-*.txt'):
    outfile = str.replace(str(text_file), '.txt', '.json')
    if os.path.exists(outfile):
        continue
    with open(text_file, 'r') as file:
        file_contents = file.read()
    parsed = simple_parse_xml(file_contents, nested=False)
    output.append({
        'file': str(text_file),
        'split': [p[0] for p in parsed[:-1]] + [parsed[-1]],
        'len': len(parsed),
        'ok': len(parsed) == 11
    })
    if len(parsed) != 11:
        omitted.append(text_file)
        continue
    with open(outfile, 'w') as file:
        file.write(json.dumps([p[1:] for p in parsed]))

with open('parse3.log', 'w') as file:
    file.write(json.dumps(output))
for out in output:
    if not out['ok']:
        continue
    print(out['file'])
    print(out['split'])

In [ ]:
with get_cursor() as cursor:
    # cursor.execute('drop table if exists snippets')
    # cursor.execute('drop table if exists region')
    cursor.execute('create table if not exists snippets (ID INTEGER PRIMARY KEY, idx INTEGER, code TEXT, locations JSON, regions JSON, SRP boolean)')
    cursor.execute('create table if not exists region (ID INTEGER PRIMARY KEY, code TEXT, vector JSON)')


In [6]:
def insert_region(code):
    with get_cursor() as cursor:
        cursor.execute('insert into region (code) values (?)', (code, ))
        return cursor.lastrowid

def insert_snippet(id, index, code, locations, regions, srp):
    with get_cursor() as cursor:
        cursor.execute(
            'insert into snippets (ID, idx, code, locations, regions, SRP) values (?, ?, ?, ?, ?, ?)',
            (id, index, code, json.dumps(locations), json.dumps(regions), srp))

def flat_wrong_tags(tags, code, m):
    clean_tags = []
    for tag in tags:
        clean_tags += flat_wrong_tag(tag, clean_tags, code, m)
    return clean_tags

def flat_wrong_tag(tag, clean_tags, code, m):
    name, open, close, sub_tags = tag
    clean_sub_tags = flat_wrong_tags(sub_tags, code, m)
    if m.group(open):
        if len(clean_tags):
            pname, popen, pclose, psub_tags = clean_tags[-1]
            if not m.group(pclose):
                clean_tags[-1] = (pname, popen, close, psub_tags)
        return clean_sub_tags
    return [(name, open, close, clean_sub_tags)]

def tag_to_json(tag, code, m, handle_region):
    name, open, close, sub_tags = tag
    start = m.span(open)[0]
    regions, body, end = tags_to_json(start, sub_tags, code, m, handle_region)
    body += code[end:m.span(close)[0]] 
    region_id = handle_region(f'function {name} () {{\n{body}\n}}')
    regions = [(start, region_id)] + regions
    return regions


def tags_to_json(outer_index, tags, code, m, handle_region):
    regions = []
    outer_body = ''
    for tag in tags:
        name, open, close, _ = tag
        outer_body += code[outer_index:m.span(open)[0]] + '\n' + name + '();\n'
        outer_index = m.span(close)[0]
        regions += tag_to_json(tag, code, m, handle_region)
    return regions, outer_body, outer_index


def to_json(tags, code, m, handle_region):
    regions, body, end = tags_to_json(0, tags, code, m, handle_region)
    body += code[end:] 
    region_id = handle_region(body)
    regions = [(0, region_id)] + regions
    if len(regions) > 1:
        regions.append((end, region_id))
    return regions

def strip_js_comments(js_code):
    js_code = re.sub(r'\n?//.*?\n', '\n', js_code)
    js_code = re.sub(r'/\*.*?\*/', '', js_code, flags=re.DOTALL)
    return js_code    


In [8]:
len(list(js_dir.glob('thread2-*.json')))/len(list(js_dir.glob('thread2-*.txt')))

0.8780487804878049

In [ ]:
n_ok, n_all = 0, 0
for text_file in tqdm(list(js_dir.glob('thread2-*.json'))):
    index = int(re.match('.*thread2-(.*)\.json', str(text_file)).group(1))
    limit = 10
    with get_cursor() as cursor:
        codes = list(cursor.execute('select id, code from shuffled limit ? offset ?', (limit, limit * index + 1)))
    with open(text_file, 'r') as file:
        file_contents = file.read()
    for (id, code), obj in zip(codes, json.loads(file_contents)):
        original = strip_js_comments(code)
        xml = obj[1]
        reg_str, tags = make_regex(simple_parse_xml(xml, True), [0])
        reg = regex.compile(reg_str, flags = regex.DOTALL)
        m = reg.match(original)
        n_all += 1
        if m:
            n_ok += 1
            tags = flat_wrong_tags(tags, original, m)
            if len(tags) == 1 and not len(tags[0][3]):
                tags = []
            regions = to_json(tags, original, m, insert_region)
            insert_snippet(id, index, original, *zip(*regions), len(tags) == 0)
            
            
print(f'{n_ok}/{n_all}')

In [ ]:
with get_cursor() as cursor:
    print(list(cursor.execute('select idx < 650, count(*), SRP from snippets group by idx < 650, SRP')))
    print(list(cursor.execute('select count(*) from region')))

In [10]:
from transformers import RobertaTokenizerFast, RobertaModel
import torch

# Load tokenizer and model
tokenizer = RobertaTokenizerFast.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base")


Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def get_labels(ids):
    with get_cursor() as cursor:
        regions = list(cursor.execute(f'select id, code from region where id in ({ids})'))
    for region in regions:
        id, code = region
        tokenized_inputs = tokenizer([code], padding=True, truncation=True, return_tensors="pt")
        tokenized_inputs.to(device)
        with torch.no_grad():
            outputs = model(**tokenized_inputs)
        last_hidden_states = outputs.last_hidden_state
        #average_hidden_states = last_hidden_states.mean(dim=1)
        yield id, list(last_hidden_states.cpu().numpy()[0][0])


In [61]:
with get_cursor() as cursor:
    print(next(cursor.execute('select count(*) from snippets'))[0])

6247


In [ ]:
with get_cursor() as cursor:
    cursor.execute('drop table if exists tokenized')
    cursor.execute('create table if not exists tokenized (ID INTEGER PRIMARY KEY, input_ids JSON, region_ids JSON)')


In [ ]:
with get_cursor() as cursor:
    snippets = list(cursor.execute('select id, code, locations, regions from snippets where idx < 650'))
for (id, code, locations, regions) in tqdm(snippets):
    tokens = tokenizer.encode_plus(code, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokens['offset_mapping']
    regions, locations = json.loads(regions),json.loads(locations)
    if not len(regions):
        continue
    locations.append(len(code))
    i = 0
    region_ids = []
    for (start, end) in offset_mapping[1:-1]:
        while start > locations[i+1]:
            i += 1
        region_ids.append(regions[i])
    region_ids = [0] + region_ids + [0]
    with get_cursor() as cursor:
        cursor.execute(
            'insert into tokenized (ID, input_ids, region_ids) values (?, ?, ?)',
            (id, json.dumps(tokens['input_ids']), json.dumps(region_ids)))


In [ ]:
with get_cursor() as cursor:
    cursor.execute('drop table if exists train_snippets')
    cursor.execute('create table train_snippets as select * from snippets where idx < 650')

In [ ]:
with get_cursor() as cursor:
    cursor.execute('create table if not exists current_batch(idx INTEGER)')
    if not len(list(cursor.execute('select * from current_batch'))):
        

In [32]:
with get_cursor() as cursor:
    cursor.execute('create table if not exists last_batch(idx INTEGER, epoch_idx INTEGER)')


In [29]:
import torch
device = torch.device('cuda')

def set_last_batch(last_batch, epoch_idx):
    with get_cursor() as cursor:
        cursor.execute('update last_batch set idx = ?, epoch_idx = ?', (last_batch, epoch_idx))

def get_unsafe_last_batch():
    with get_cursor() as cursor:
        cursor.execute('select * from last_batch')
        item = cursor.fetchone()
        if not item:
            cursor.execute('insert into last_batch values (0, 0)')
            return 0
        else:
            return item

def get_last_batch(num_batches):
    last_batch, last_epoch = get_unsafe_last_batch()
    if last_batch >= num_batches:
        last_batch = 0
        last_epoch += 1
        set_last_batch(0, last_epoch)
    return last_batch, last_epoch
        
def get_batch_tokens(batch_size, i):
    with get_cursor() as cursor:
        ids = ','.join(
            str(x[0]) for x in cursor.execute('select id from train_snippets_shuffled limit ? offset ?', (batch_size, i * batch_size))
        )
        tokens = cursor.execute(f'select input_ids, region_ids from tokenized where id in ({ids})')
        return [(json.loads(input_ids), json.loads(region_ids)) for (input_ids, region_ids) in tokens]
        

def get_batch_regions(tokens):
    with get_cursor() as cursor:
        region_ids_str = ','.join(set(str(region_id) for (_, region_ids) in tokens for region_id in region_ids))
        return dict(get_labels(region_ids_str))

def prepare_iteration(input_ids, region_ids, regions):
    label_size = 768
    size = len(input_ids)
    if size > 512:
        print(size)
    empties = 512 - size
    null_vector = [0] * label_size
    for r_id in region_ids:
        if r_id and not (r_id in regions):
            print(r_id)
    label = [regions[r_id] if r_id else null_vector for r_id in region_ids]
    label += [null_vector] * empties
    input_ids += [0] * empties
    attention = [1] * size + [0] * empties
    try:
        torch.FloatTensor(label)
    except:
        print(regions[region_ids[1]])

    return input_ids, attention, label

def get_batch_data(batch_size, i):
    tokens = get_batch_tokens(batch_size, i)
    regions = get_batch_regions(tokens)
    batch = [prepare_iteration(input_ids, region_ids, regions) for (input_ids, region_ids) in tokens]
    input_ids, attention, label = zip(*batch)
    return (torch.IntTensor(input_ids), torch.FloatTensor(attention), torch.FloatTensor(label)), tokens


def get_epoch_part(i0, num_batches, batch_size):
    return ((i, get_batch_data(batch_size, i)) for i in range(i0, num_batches, 1))

def tee(text):
    print(f"\r{text}\r")
    with open('train-resnet.log', 'a') as file:
        file.write(f"{text}\n")

def log(title, start_time, sizes, epoch, i, mean_loss, loss):
    _, num_epochs, num_batches = sizes
    i += 1
    dt = time.time() - start_time
    elapsed = timedelta(seconds=int(dt))
    remaining = timedelta(seconds=int(dt*(num_batches-i)/i))
    text = f"{title}: {elapsed}<{remaining} Epoch {epoch+1}/{num_epochs} - Batch {i}/{num_batches}, Loss: {mean_loss:.4f} {loss.item():.4f}"
    tee(text)

def save(epoch, i):
    torch.save(model.state_dict(), f'searchnet-bcemodel-{epoch}-{i}.pt')
    torch.save(optimizer.state_dict(), f'searchnet-bceoptimizer-{epoch}-{i}.pt')
    set_last_batch(i, epoch)
    tee(f'saving searchnet-bcemodel-{epoch}-{i}.pt\n')

def compute_loss(batch_data):
    batch_vectors, batch_tokens = batch_data
    input_ids, attention, labels = (c.to(device) for c in batch_vectors)

    outputs = model(input_ids, attention_mask=attention)

    region_ids = torch.tensor([region_ids + [0] * (512 - len(region_ids)) for (input_ids, region_ids) in batch_tokens])
    padding_mask = (region_ids == 0).to(device)
    labels[padding_mask] = outputs.last_hidden_state[padding_mask]

    
    return loss_fn(outputs.last_hidden_state, labels)


def handle_train_batch(i, epoch, batch_data, performance, sizes):

    optimizer.zero_grad()
    loss = compute_loss(batch_data)
    loss.backward()
    optimizer.step()
    
    min_alpha, alpha, running_loss, start_time = performance

    current_alpha = min_alpha + alpha
    running_loss = (1 - current_alpha) * running_loss + current_alpha * loss.item()
    alpha *= .5
    
    log('training', start_time, sizes, epoch, i, running_loss, loss)
    if (i+1) % 100 == 0:
        save(epoch, i)
    return running_loss, alpha
    
def handle_val_batch(i0, i, epoch, batch_data, start_time, cum_loss, sizes):

    with torch.no_grad():
        loss = compute_loss(batch_data)
    
    cum_loss += loss.item()
    mean_loss = cum_loss / (i - i0 + 1)
    
    log('validation', start_time, sizes, epoch, i, mean_loss, loss)
    return cum_loss
    

def handle_epoch(batch_size, num_epochs):
    with get_cursor() as cursor:
        epoch_size = next(cursor.execute('select count(*) from train_snippets'))[0]
    num_batches = (epoch_size - 1) // batch_size + 1
    train_batchs = 4 * num_batches // 5
    i0, epoch0 = get_last_batch(train_batchs)
    num_epochs += epoch0
    i = i0 - 1
    min_alpha = .2
    start_time = time.time()
    
    
    running_loss = 0.0
    alpha = 1 - min_alpha

    model.train()
    sizes = (train_batchs, num_epochs, num_batches)
    tee('\nTraining\n')
    for (i, batch_data) in get_epoch_part(i0, train_batchs, batch_size):
        performance = (min_alpha, alpha, running_loss, start_time)
        running_loss, alpha = handle_train_batch(i, epoch, batch_data, performance, sizes)
    
    
    i0 = i + 1
    save(epoch, i0)
    
    model.eval()
    cum_loss = 0
    tee('\nValidation:\n')
    for (i, batch_data) in get_epoch_part(i0, num_batches, batch_size):
        cum_loss = handle_val_batch(i0, i, epoch, batch_data, start_time, cum_loss, sizes)
    
        
        


In [30]:
#set_last_batch(0, 0)

In [31]:
from transformers import RobertaTokenizerFast, RobertaModel
import torch
import torch.nn as nn
from torch.optim import AdamW
import time
from datetime import timedelta

loss_fn = nn.MSELoss()

device = torch.device('cuda')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 10
batch_size = 16

set_last_batch(0, 0)
for epoch in range(num_epochs):
    handle_epoch(batch_size, num_epochs)

#model.load_state_dict(torch.load('searchnet-bcemodel-0-200.pt'))



Training

training: 0:00:27<2:27:32 Epoch 1/10 - Batch 1/320, Loss: 0.1226 0.1226
training: 0:00:50<2:13:25 Epoch 1/10 - Batch 2/320, Loss: 0.1353 0.1437
training: 0:01:11<2:06:34 Epoch 1/10 - Batch 3/320, Loss: 0.1489 0.1694
training: 0:01:33<2:03:26 Epoch 1/10 - Batch 4/320, Loss: 0.1366 0.1078
training: 0:01:56<2:01:53 Epoch 1/10 - Batch 5/320, Loss: 0.1379 0.1420
training: 0:02:17<1:59:56 Epoch 1/10 - Batch 6/320, Loss: 0.1306 0.1051
training: 0:02:39<1:58:55 Epoch 1/10 - Batch 7/320, Loss: 0.1255 0.1069
training: 0:03:01<1:57:51 Epoch 1/10 - Batch 8/320, Loss: 0.1145 0.0720
training: 0:03:23<1:56:56 Epoch 1/10 - Batch 9/320, Loss: 0.1057 0.0714
training: 0:03:44<1:55:53 Epoch 1/10 - Batch 10/320, Loss: 0.0997 0.0758
training: 0:04:05<1:54:49 Epoch 1/10 - Batch 11/320, Loss: 0.0934 0.0685
training: 0:04:26<1:53:53 Epoch 1/10 - Batch 12/320, Loss: 0.0851 0.0518
training: 0:04:48<1:53:21 Epoch 1/10 - Batch 13/320, Loss: 0.0786 0.0529
training: 0:05:08<1:52:29 Epoch 1/10 - Batch 14/3

KeyboardInterrupt: 

In [33]:
set_last_batch(0, 0)

In [11]:
from transformers import RobertaTokenizerFast, RobertaModel
import torch
device = torch.device('cuda')
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base").to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import pdist

# Function to find the best matching between predicted and true labels
def best_label_matching(y_true, y_pred):
    D = np.max([y_pred.max(), y_true.max()]) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return list(zip(row_ind, col_ind))



def compute_accuracy(data, true_labels):


    unique_labels = np.unique(true_labels)
    label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
    true_labels = np.array([label_mapping[label] for label in true_labels])

    
    # Perform Hierarchical Agglomerative Clustering
    Z = linkage(pdist(data), method='ward')
    predicted_labels = fcluster(Z, t=len(set(true_labels)), criterion='maxclust')
    
    
    # Find the best matching
    ind = best_label_matching(true_labels, predicted_labels)
    ind_dict = {i[0]: i[1] for i in ind}
    
    # Remap predicted labels to best matching true labels
    remapped_predicted_labels = np.array([ind_dict.get(label, 0) for label in predicted_labels])
    
    # Calculate accuracy
    return accuracy_score(true_labels, remapped_predicted_labels)
    



In [24]:
data = np.random.rand(512, 786)  # Replace with your actual data
true_labels = np.random.randint(5, size=512)  # Replace with your actual labels

In [26]:
print(f"Accuracy: {compute_accuracy(data, true_labels+10)}")

Accuracy: 0.26171875


In [76]:
import numpy as np
d = []
def get_batcgh(batch_size, i):
    batch, tokens = get_batch_data(batch_size, i)
    input_ids, attention, label = (c.to(device) for c in batch)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention)[0]
        data = outputs.cpu()[0].numpy()
        true_labels = np.array(tokens[0][1])
        size = len(true_labels[1:-1])
        
        data = data[1:size]
        true_labels = true_labels[1:size]
        
        unique_labels = np.unique(true_labels)
        label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        true_labels = np.array([label_mapping[label] for label in true_labels])
        
        Z = linkage(pdist(data), method='ward')
        
        predicted_labels = fcluster(Z, t=len(set(true_labels)), criterion='maxclust')
        
        ind = best_label_matching(true_labels, predicted_labels)
        ind_dict = {i[0]: i[1] for i in ind}
        
        remapped_predicted_labels = np.array([ind_dict.get(label, 0) for label in predicted_labels])
        return accuracy_score(true_labels, remapped_predicted_labels)

get_batcgh(1, 43)

1.0

In [80]:
c = [get_batcgh(1, x) for x in range(30)]

In [82]:
c = [x for x in c if x != 1]

In [83]:
sum(c)/len(c)

0.4781032156239851

In [70]:
(data, true_labels, input_ids), = d

In [75]:
size = len(true_labels[1:-1])

data = data[1:size]
true_labels = true_labels[1:size]

unique_labels = np.unique(true_labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
true_labels = np.array([label_mapping[label] for label in true_labels])

Z = linkage(pdist(data), method='ward')

predicted_labels = fcluster(Z, t=len(set(true_labels)), criterion='maxclust')

ind = best_label_matching(true_labels, predicted_labels)
ind_dict = {i[0]: i[1] for i in ind}

remapped_predicted_labels = np.array([ind_dict.get(label, 0) for label in predicted_labels])
accuracy_score(true_labels, remapped_predicted_labels)

1.0

In [60]:
data = data[1:57]
true_labels = true_labels[1:57]

In [61]:
unique_labels = np.unique(true_labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
true_labels = np.array([label_mapping[label] for label in true_labels])


In [39]:
len(true_labels[:58])

58

In [62]:
Z = linkage(pdist(data), method='ward')


In [63]:
predicted_labels = fcluster(Z, t=len(set(true_labels)), criterion='maxclust')
predicted_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [64]:
true_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [65]:

ind = best_label_matching(true_labels, predicted_labels)
ind_dict = {i[0]: i[1] for i in ind}



In [66]:
remapped_predicted_labels = np.array([ind_dict.get(label, 0) for label in predicted_labels])



In [67]:
remapped_predicted_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
accuracy_score(true_labels, remapped_predicted_labels)

1.0

In [58]:
class CodeSegmentationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.codebert = RobertaModel.from_pretrained("microsoft/graphcodebert-base")
        self.conv = nn.Conv1d(in_channels=768, out_channels=256, kernel_size=3, padding=1)
        self.bilstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, input_ids, attention_mask):
        # Get embeddings from CodeBERT
        embeddings = self.codebert(input_ids, attention_mask=attention_mask).last_hidden_state
        
        # Mask padding embeddings
        padding_mask = (input_ids != padding_token_id)  # Define padding_token_id
        embeddings[~padding_mask] = 0

        # Pass through additional layers
        embeddings = embeddings.transpose(1, 2)  # Needed for Conv1d
        conv_out = self.conv(embeddings).transpose(1, 2)
        lstm_out, _ = self.bilstm(conv_out)
        # Further processing and return

model = CodeSegmentationModel(model)  # Initialize with pre-trained CodeBERT


In [1]:
model

NameError: name 'model' is not defined